### 1) Is there a native array data structure in python?
Yes, in Python 3 there are two:
 - [Array module](https://docs.python.org/3/library/array.html) for representing an array of basic values: characters, integers, floating point numbers.
 - Built-in [bytearray](https://docs.python.org/3/library/stdtypes.html#bytearray) data structure. Limited to holding bytes.

These are **not popular** because of the flexibility and features of the [numpy package](https://docs.scipy.org/doc/numpy-1.13.0/user/whatisnumpy.html) and [built-in lists](https://docs.python.org/3/library/stdtypes.html#lists) which provide much greater functionality for arrays-like objects.  
It is good to remember that arrays are designed to hold **homogeneous** data. For arrays to be efficient, you would have to know the array size in advance to reserve a block of memory for the array. 
Lists in pythons are implemented as arrays for **homogeneous** data and they are also efficient if you know the size of your list in advance (see below). If the number of items in your array keeps growing, using lists is a good approach, since the memory allocation (the process of reserving memory) is automatically done by Python.

In [1]:
import timeit
#To use time it, you give some setup statements (setup) and then include an statement to time (stmt) 

print("If you know how many items your array will hold, an array is a very efficient data structure. It takes only",
      "%.2f" % timeit.timeit(stmt='array1 = array.array("B",[128])*N', setup='N = 10**8; import array',number=1),
     "seconds fill 10^8 items in an array.")

print("If you know the number of items, a numpy array is also a very efficient data structure. It takes only",
      "%.2f" % timeit.timeit(stmt='array2 = np.full((N,1), 128)', setup='N = 10**8; import numpy as np',number=1),
     "seconds to fill 10^8 items in a numpy array")

print("If you know the number of items, a list is also an efficient data structure. It takes only",
      "%.2f" % timeit.timeit(stmt='list1 = [128]*N', setup='N = 10**8; list1 = list()',number=1),
     "seconds to fill 10^8 items in a list")

print("If you do not know the number of items, a list is less efficient, because it will have to resize itself.",
      "The resizing time is not an issue for small lists, but it could be a problem for large lists. It takes",
      "%.2f" % timeit.timeit(stmt='list2.append(128)', setup='list2 = list()',number=10**8),
     "seconds for appending 10^8 items")

If you know how many items your array will hold, an array is a very efficient data structure. It takes only 0.08 seconds fill 10^8 items in an array.
If you know the number of items, a numpy array is also a very efficient data structure. It takes only 0.19 seconds to fill 10^8 items in a numpy array
If you know the number of items, a list is also an efficient data structure. It takes only 0.46 seconds to fill 10^8 items in a list
If you do not know the number of items, a list is less efficient, because it will have to resize itself. The resizing time is not an issue for small lists, but it could be a problem for large lists. It takes 10.74 seconds for appending 10^8 items


### 2) Are lists in Python implemented as linked lists?
No, Python lists are not linked lists, they are [equivalent to arrays](https://docs.python.org/3/faq/programming.html#how-do-you-make-an-array-in-python) in their performance. Most of the time they will use contiguous memory blocks for fast indexing. However, they are **dynamically allocated** arrays, which means that you do not have to worry about reserving memory for the elements in this data structure, and also they can be used to store **heterogeneous** data. 

### 3) Should we always avoid for loops?
This is a very complex question that is more relevant to software engineers. At the end, many algorithms use for loops under the hood, but they use it with the most basic data structures implemented with optimized libraries.  
When we use for loops in our code, we are using the python high level version of a for loop, and this for loop may not be able to access the optimized functions. 
So **if there is a vectorized or optimized version of a function** in Python, use it instead of a for loop. Sometimes a for loop can be replaced by a [map call or a list comprehension](https://wiki.python.org/moin/PythonSpeed/PerformanceTips#Loops).  

In [3]:
print("To fill in a list with the value 128 in 100 million locations, using a Python shorcut takes only",
      "%.2f" % timeit.timeit(stmt='list1 = [128]*N', setup='N = 10**8; import array',number=1))
print("Filling in the list with a for loop is slower, taking",
      "%.2f" % timeit.timeit(stmt='for i in range(N): list1[i] = 128', setup='N = 10**8; list1 = [None]*N',number=1))

To fill in a list with the value 128 in 100 million locations, using a Python shorcut takes only 0.50
Filling in the list with a for loop is slower, taking 5.08


Here is another example of timing for loops versus an optimized numpy function for multiplying matrices. Although Numpy may use a for loop in the background, they use it with [special optimized libraries](https://stackoverflow.com/questions/43857033/fastest-way-to-compute-matrix-dot-product) and simple basic data types. Fortunately as Data Scientist we do not have to develop those libraries, but it is good to know they exist. 

In [4]:
# Matrix multiplication with brute force for loops O(n^3)
matmul = """
result = [] # final result
for i in range(len(matrix1)):
    row = [] # the new row in new matrix
    for j in range(len(matrix2)):
        product = 0 # the new element in the new row
        for v in range(len(matrix1[i])):
            product += matrix1[i][v] * matrix2[v][j]
        row.append(product) # append sum of product into the new row
    result.append(row) # append the new row into the final result
    """

timeit.timeit(stmt=matmul,
              setup='matrix1 = [[9]*10**2]*10**2;matrix2 = [[11]*10**2]*10**2; import numpy as np',
              number=1)


0.2566298488737715

In [5]:
# Optimized matrix multiplication with numpy
timeit.timeit(stmt='res = np.dot(matrix1,matrix2)',
              setup='matrix1 = [[9]*10**2]*10**2;matrix2 = [[11]*10**2]*10**2; import numpy as np',
              number=1)

0.019385443850438833

### 4) When to use linked lists?
A linked list can be used when you want inexpensive insertion and deletion of elements and when it doesn't matter that the elements aren't next to each other in memory. 
However, the inexpensive insertion/deletion comes to the expense of linear time complexity (O(n)) retrieval of elements. Here is a decent [Medium post](https://medium.com/@kojinoshiba/data-structures-in-python-series-1-linked-lists-d9f848537b4d) about linked lists. 

### 5) Clarification about the answer to finding a median in a stream of integers with two heaps
The problem definition can be found in [LeetCode](https://leetcode.com/problems/find-median-from-data-stream/description/)  

Given that integers are read from a data stream. Find median of elements in real-time in an efficient way. 

For example, consider the stream 5, 15, 1, 3 …
After reading 1st element of stream - 5 -> median - 5
After reading 2nd element of stream - 5, 15 -> median - 10 
After reading 3rd element of stream - 5, 15, 1 -> median - 5
After reading 4th element of stream - 5, 15, 1, 3 -> median - 4

A discussion about the solution (including Python code) can be found [here](http://www.ardendertat.com/2011/11/03/programming-interview-questions-13-median-of-integer-stream/). Note that they use the Python implementation of heaps (heapq).

In [6]:
import heapq
class streamMedian:
    def __init__(self):
        self.minHeap, self.maxHeap = [], []
        self.N=0
 
    def insert(self, num):
        if self.N%2==0:
            heapq.heappush(self.maxHeap, -1*num)
            self.N+=1
            if len(self.minHeap)==0:
                return
            if -1*self.maxHeap[0]>self.minHeap[0]:
                toMin=-1*heapq.heappop(self.maxHeap)
                toMax=heapq.heappop(self.minHeap)
                heapq.heappush(self.maxHeap, -1*toMax)
                heapq.heappush(self.minHeap, toMin)
        else:
            toMin=-1*heapq.heappushpop(self.maxHeap, -1*num)
            heapq.heappush(self.minHeap, toMin)
            self.N+=1
 
    def getMedian(self):
        if self.N%2==0:
            return (-1*self.maxHeap[0]+self.minHeap[0])/2.0
        else:
            return -1*self.maxHeap[0]

In [7]:
myStream = streamMedian()
myStream.insert(5)
print("median -",myStream.getMedian())

median - 5


In [8]:
myStream.insert(15)
print("median -",myStream.getMedian())

median - 10.0


In [9]:
myStream.insert(1)
print("median -",myStream.getMedian())

median - 5


In [10]:
myStream.insert(3)
print("median -",myStream.getMedian())

median - 4.0
